In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from hyperopt import fmin, tpe, hp, STATUS_OK
from hyperopt import SparkTrials
import mlflow

In [2]:
import mlflow
import time
mlflow_uri = "http://mlflow-server:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
mlflow.set_experiment("sample_ds_training")


2024/10/31 17:21:25 INFO mlflow.tracking.fluent: Experiment with name 'sample_ds_training' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/290695839892204708', creation_time=1730395285883, experiment_id='290695839892204708', last_update_time=1730395285883, lifecycle_stage='active', name='sample_ds_training', tags={}>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sample_ds_training").master("spark://spark-master:7077") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 17:21:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/31 17:21:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def optimize(C):
    clf = SVC(C = C)

    # Use the cross validation object to compare the model's performance
    accuracy = cross_val_score(clf, X, y).mean()

    return {'loss': -accuracy, 'status': STATUS_OK}

In [5]:
search_space = hp.lognormal('C', 0 , 1.0)
algo = tpe.suggest

In [6]:
from hyperopt import SparkTrials
spark_trials = SparkTrials()

Because the requested parallelism was None or a non-positive value, parallelism will be set to (2), which is Spark's default parallelism (2), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


In [ ]:
with mlflow.start_run(run_name='hyperopt') as run:
    argmin = fmin(
    fn= optimize,
    space=search_space,
    algo=algo,
    max_evals = 3,
    trials=spark_trials)

2024/10/31 17:21:27 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



  0%|          | 0/16 [00:00<?, ?trial/s, best loss=?]

24/10/31 17:21:44 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
